In [1]:
from mbientlab.warble import *
from mbientlab.metawear import *
from ctypes import cast, byref
from threading import Event
from time import sleep
import pandas as pd
from math import nan

In [3]:
e = Event()
address = None
def device_discover_task(result):
    global address
    if (result.has_service_uuid(MetaWear.GATT_SERVICE)):
        # grab the first discovered metawear device
        address = result.mac
        e.set()

BleScanner.set_handler(device_discover_task)
BleScanner.start()
e.wait()

BleScanner.stop()

In [11]:
# Initialize d and connect
d = MetaWear(address)
d.connect()
e = Event()

In [12]:
freq = 100
acc_range = 16
gyro_range = 2000

libmetawear.mbl_mw_acc_set_odr(d.board, freq)
libmetawear.mbl_mw_acc_set_range(d.board, acc_range)
libmetawear.mbl_mw_acc_write_acceleration_config(d.board)

if freq == 800:
    gyro_freq = GyroBoschOdr._800Hz
elif freq == 400:
    gyro_freq = GyroBoschOdr._400Hz
elif freq == 200:
    gyro_freq = GyroBoschOdr._200Hz
elif freq == 100:
    gyro_freq = GyroBoschOdr._100Hz
elif freq == 50:
    gyro_freq = GyroBoschOdr._50Hz
elif freq == 25:
    gyro_freq = GyroBoschOdr._25Hz
else:
    gyro_freq = GyroBoschOdr._100Hz
    
if gyro_range == 2000:
    gyro_ra = GyroBoschRange._2000dps
elif gyro_range == 1000:
    gyro_ra = GyroBoschRange._1000dps
elif gyro_range == 500:
    gyro_ra = GyroBoschRange._500dps
elif gyro_range == 250:
    gyro_ra = GyroBoschRange._250dps
elif gyro_range == 125:
    gyro_ra = GyroBoschRange._125dps
else:
    gyro_ra = GyroBoschRange._2000dps
    
libmetawear.mbl_mw_gyro_bmi160_set_odr(d.board, gyro_freq)
libmetawear.mbl_mw_gyro_bmi160_set_range(d.board, gyro_ra)
libmetawear.mbl_mw_gyro_bmi160_write_config(d.board)

# Acc and gyro signals
acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(d.board)
gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(d.board)

# Create a logger
acc_logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(acc, None, fn), resource = "acc_logger")
gyro_logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(gyro, None, fn), resource = "gyro_logger")

# Start logger
libmetawear.mbl_mw_logging_start(d.board, 0)
# Turn on the accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(d.board)
libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(d.board)
libmetawear.mbl_mw_acc_start(d.board)
libmetawear.mbl_mw_gyro_bmi160_start(d.board)


print("Logging data for 5s")
sleep(5.0)

# Turn off the accelerometer
libmetawear.mbl_mw_acc_stop(d.board)
libmetawear.mbl_mw_gyro_bmi160_stop(d.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(d.board)
libmetawear.mbl_mw_gyro_bmi160_disable_rotation_sampling(d.board)
# Stop logging
libmetawear.mbl_mw_logging_stop(d.board)

print("Downloading data")
# Callback function to handle logger entries as we download them
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()

# Function pointer and handlers for the logger download (LogDownloadHandler -> in the cbindings)
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler)
download_handler = LogDownloadHandler(context = None, received_progress_update = fn_wrapper, received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))

acc_data = {}
gyro_data = {}

def acc_data_handler(context, p):
    parsed = parse_value(p)
    acc_data[int(p.contents.epoch)] = {'x': parsed.x, 'y': parsed.y, 'z': parsed.z}
    print(f'Sensor: acc, epoch: {p.contents.epoch}, values: {parsed}')

def gyro_data_handler(context, p):
    parsed = parse_value(p)
    gyro_data[int(p.contents.epoch)] = {'x': parsed.x, 'y': parsed.y, 'z': parsed.z}
    print(f'Sensor: gyro, epoch: {p.contents.epoch}, values: {parse_value(p)}')
    
acc_callback = FnVoid_VoidP_DataP(acc_data_handler)
gyro_callback = FnVoid_VoidP_DataP(gyro_data_handler)

# Stop logger
libmetawear.mbl_mw_logger_subscribe(acc_logger, None, acc_callback)
libmetawear.mbl_mw_logger_subscribe(gyro_logger, None, gyro_callback)

# Download logger contents
libmetawear.mbl_mw_logging_download(d.board, 0, byref(download_handler))
e.wait()

libmetawear.mbl_mw_logging_clear_entries(d.board)
libmetawear.mbl_mw_debug_reset(d.board)

Logging data for 5s
Sensor: acc, epoch: 1714380949166, values: {x : -0.011, y : -1.065, z : 0.001}
Sensor: gyro, epoch: 1714380949166, values: {x : -0.366, y : 0.122, z : -0.366}
Sensor: acc, epoch: 1714380949175, values: {x : -0.012, y : -1.073, z : 0.002}
Sensor: gyro, epoch: 1714380949175, values: {x : -0.305, y : 0.183, z : 0.305}
Sensor: acc, epoch: 1714380949185, values: {x : -0.012, y : -1.071, z : -0.001}
Sensor: gyro, epoch: 1714380949185, values: {x : -0.183, y : 0.549, z : 0.061}
Sensor: acc, epoch: 1714380949196, values: {x : -0.014, y : -1.072, z : -0.003}
Sensor: gyro, epoch: 1714380949196, values: {x : -0.122, y : 0.549, z : 0.061}
Sensor: acc, epoch: 1714380949206, values: {x : -0.013, y : -1.072, z : 0.001}
Sensor: gyro, epoch: 1714380949206, values: {x : -0.122, y : 0.488, z : 0.122}
Sensor: acc, epoch: 1714380949216, values: {x : -0.014, y : -1.070, z : 0.001}
Sensor: gyro, epoch: 1714380949216, values: {x : -0.305, y : 0.488, z : 0.000}
Sensor: acc, epoch: 171438094

In [10]:
all_epochs = sorted(set(list(acc_data.keys()) + list(gyro_data.keys())))
data_df = pd.DataFrame(columns=['Timestamp', 'x_acc', 'y_acc', 'z_acc', 'x_gyro', 'y_gyro', 'z_gyro'])

for epoch in all_epochs:
    row = [float(epoch - all_epochs[0])]
    if epoch in acc_data.keys() and epoch in gyro_data.keys():
        row += [acc_data[epoch]['x'], acc_data[epoch]['y'], acc_data[epoch]['z'], gyro_data[epoch]['x'], gyro_data[epoch]['y'], gyro_data[epoch]['z']]
    elif epoch in acc_data.keys():
        row += [acc_data[epoch]['x'], acc_data[epoch]['y'], acc_data[epoch]['z'], nan, nan, nan]
    else:
        row += [nan, nan, nan, gyro_data[epoch]['x'], gyro_data[epoch]['y'], gyro_data[epoch]['z']]
    data_df.loc[len(data_df)] = row
    
data_df.head(100)

,Timestamp,x_acc,y_acc,z_acc,x_gyro,y_gyro,z_gyro
0,0.0,-0.012695,-1.061523,0.000977,-0.548781,0.426829,0.731707
1,10.0,-0.013184,-1.074707,0.001953,-0.243902,0.731707,-0.060976
2,21.0,-0.012207,-1.070801,0.000000,-0.121951,0.487805,-0.121951
3,31.0,-0.013672,-1.071777,-0.002441,-0.304878,0.365854,0.182927
4,40.0,-0.012695,-1.073242,-0.004883,-0.304878,0.426829,0.182927
...,...,...,...,...,...,...,...
95,935.0,-0.011719,-1.071777,-0.000488,NaN,NaN,NaN
96,936.0,NaN,NaN,NaN,-0.182927,0.487805,0.060976
97,945.0,-0.012695,-1.073242,0.002441,-0.182927,0.487805,0.060976
98,955.0,-0.013184,-1.073730,-0.002441,-0.182927,0.548781,0.060976
